In [1]:
import os
import csv
import pandas as pd
import numpy as np

In [2]:
if not os.path.exists('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv'):
        
    (
        pd.read_csv('../input_data_clean/census_clean.csv')
        .merge(pd.read_csv('../input_data_clean/bank_tract_clean.csv'),
               on = "census_tract", 
               how = "inner",
               validate = "1:m")
        .to_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')
    )

In [3]:
data = pd.read_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')

In [4]:
data.describe()
#data.head()

,Unnamed: 0,Tot.Pop,Tot.WhitePop,Tot.BlackPop,Tot.AmericIndianPop,Tot.AsianPop,Tot.NativeHawaiianPacificPop,Tot.OtherRaceAlonePop,Tot.TwoOrMoreRace,Tot.TwoOrMoreRace(Some other race),...,census_tract,num_approved_loans,num_applications,sum_all_loan_amount,sum_rate_spread,median__all_income,mean_approved_loans,mean_LTV,mean_approved_spread,mean_approved_interest_rate
count,15698.000000,15698.000000,15698.000000,15698.000000,15698.000000,15698.000000,15698.000000,15698.000000,15698.000000,15698.000000,...,1.569800e+04,15698.000000,15698.000000,1.569800e+04,15698.000000,1.524300e+04,1.463000e+04,14605.000000,14049.000000,14630.000000
mean,7848.500000,5002.376545,3163.959867,267.261243,55.709836,669.756466,18.261689,587.331826,240.095617,49.950631,...,5.707714e+09,162.316282,188.442604,8.526094e+07,41.543092,2.593726e+02,4.852678e+05,143.340828,0.435248,3.204030
std,4531.766598,2285.087213,1752.529285,419.573251,223.035024,969.694268,54.148746,744.078790,203.137291,67.925071,...,7.670065e+08,228.374770,261.618564,1.346189e+08,73.957674,1.296242e+04,9.473311e+05,4636.250615,1.827695,0.542876
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.001943e+09,0.000000,1.000000,1.500000e+04,-105.000610,0.000000e+00,1.500000e+04,2.884700,-1.095000,1.583000
25%,3924.250000,3541.250000,1950.000000,38.000000,0.000000,106.000000,0.000000,101.000000,105.000000,4.000000,...,6.017031e+09,2.000000,3.000000,8.550000e+05,0.448250,8.300000e+01,2.550000e+05,56.142125,0.168093,2.950976
50%,7848.500000,4694.000000,2924.000000,120.000000,17.000000,326.000000,0.000000,301.000000,194.000000,28.000000,...,6.037621e+09,89.000000,107.000000,3.825750e+07,14.177510,1.140000e+02,3.850000e+05,63.876000,0.291170,3.038333
75%,11772.750000,6022.000000,4051.000000,318.000000,52.000000,803.000000,12.000000,783.750000,320.000000,67.000000,...,6.073000e+09,254.000000,295.000000,1.205225e+08,58.624608,1.700000e+02,5.491278e+05,70.771495,0.490000,3.243196
max,15697.000000,38754.000000,29271.000000,5801.000000,6032.000000,13013.000000,838.000000,7961.000000,2718.000000,806.000000,...,6.115041e+09,4299.000000,4881.000000,2.358025e+09,1567.285980,1.600000e+06,5.417500e+07,344253.728634,148.034667,10.179545


In [5]:
data['pct_minority'] = (data['Tot.Pop'] - data['Tot.WhitePop'])/data['Tot.Pop'] * 100
#data['pct_minority'] = data['pct_minority'].apply(lambda x: "{:.2f%}".format(x))


In [6]:
data['pct_minority'].head()

0    23.269231
1    23.269231
2    21.425012
3    21.425012
4    25.796872
Name: pct_minority, dtype: float64

In [7]:
#New dataframe that contains only the rows where the % of the non-white population is over 50%
majmin_tracts = data[data['pct_minority']>50]

In [8]:
majmin_tracts.head()

,Unnamed: 0,NAME,Tot.Pop,Tot.WhitePop,Tot.BlackPop,Tot.AmericIndianPop,Tot.AsianPop,Tot.NativeHawaiianPacificPop,Tot.OtherRaceAlonePop,Tot.TwoOrMoreRace,...,num_approved_loans,num_applications,sum_all_loan_amount,sum_rate_spread,median__all_income,mean_approved_loans,mean_LTV,mean_approved_spread,mean_approved_interest_rate,pct_minority
8,8,"Census Tract 4005, Alameda County, California",4124.0,2036.0,882.0,0.0,446.0,22.0,110.0,628.0,...,215,241,146455000.0,28.00794,160.0,625883.720930,54.866377,0.147410,3.077451,50.630456
9,9,"Census Tract 4005, Alameda County, California",4124.0,2036.0,882.0,0.0,446.0,22.0,110.0,628.0,...,3,4,520000.0,2.41500,74.0,138333.333333,43.560700,0.805000,3.635000,50.630456
11,11,"Census Tract 4007, Alameda County, California",5128.0,2183.0,1459.0,14.0,329.0,0.0,572.0,571.0,...,293,347,216075000.0,52.51778,160.5,650563.139932,54.010906,0.213487,3.125962,57.429797
12,12,"Census Tract 4007, Alameda County, California",5128.0,2183.0,1459.0,14.0,329.0,0.0,572.0,571.0,...,1,2,1110000.0,0.18000,164.0,255000.000000,77.833000,0.180000,3.000000,57.429797
13,13,"Census Tract 4008, Alameda County, California",4069.0,1827.0,1007.0,29.0,618.0,0.0,204.0,384.0,...,245,273,147085000.0,22.71791,170.5,546632.653061,59.334243,0.109221,2.970624,55.099533


In [9]:
hmda_vars = ['num_approved_loans', 
             'num_applications',
             #'median__all_income',
             #'sum_rate_spread',
             #'mean_LTV'
            ]
majmin_tracts.groupby('pct_minority')[hmda_vars].mean()

,num_approved_loans,num_applications
pct_minority,,
50.012258,99.5,119.5
50.017007,105.0,114.0
50.048631,144.0,180.5
50.064599,106.5,123.5
50.072659,71.0,85.0
...,...,...
99.458528,1.0,3.0
99.591739,1.0,5.0
99.610462,4.0,20.0


In [10]:
pivot_table = pd.pivot_table(majmin_tracts, 
                             index='pct_minority', 
                             values=hmda_vars, 
                             aggfunc='mean')

# Print the result
print(pivot_table)


              num_applications  num_approved_loans
pct_minority                                      
50.012258                119.5                99.5
50.017007                114.0               105.0
50.048631                180.5               144.0
50.064599                123.5               106.5
50.072659                 85.0                71.0
...                        ...                 ...
99.458528                  3.0                 1.0
99.591739                  5.0                 1.0
99.610462                 20.0                 4.0
99.649784                  2.0                 1.0
100.000000                 1.5                 1.0

[2554 rows x 2 columns]


In [11]:
data['majority_poor'] = data['RatioIncomeToPovertyLevel'] < 100

In [12]:
#New dataframe that contains only the rows where the ratio income to poverty is less than 100%
## which is when income is less than poverty level, therefore a poor tract (according to census)
poor_tracts = data['majority_poor']
poor_tracts.head()

0    False
1    False
2    False
3    False
4    False
Name: majority_poor, dtype: bool

In [13]:
hmda_vars = ['num_approved_loans', 
             'num_applications',
             #'median__all_income',
             #'sum_rate_spread',
             #'mean_LTV'
            ]
majmin_tracts.groupby('NAME')[hmda_vars].mean()

,num_approved_loans,num_applications
NAME,,
"Census Tract 1, Fresno County, California",2.0,5.0
"Census Tract 1, San Joaquin County, California",12.5,17.5
"Census Tract 1.02, Monterey County, California",238.5,279.0
"Census Tract 1.03, Monterey County, California",105.0,124.0
"Census Tract 1.03, San Bernardino County, California",125.5,149.0
...,...,...
"Census Tract 998.01, Orange County, California",77.5,92.0
"Census Tract 998.02, Orange County, California",76.0,96.0
"Census Tract 998.03, Orange County, California",57.5,68.5


In [27]:
# 18.9% is the national average for hispanic/latino population
# Here, we consider anything over the national average of 18.9% a high hispanic population
# Hispanic pop in AZ = 32.3%, in CA = 39.4%
data['high_hisp_pop'] = ((data['Tot.Hispanic/Latino']/data['Tot.Pop']) > 0.189)
high_hispanic= data[data['high_hisp_pop']]

In [29]:
hmda_vars = ['num_approved_loans', 
             'num_applications',
             #'median__all_income',
             #'sum_rate_spread',
             #'mean_LTV'
            ]
high_hispanic.groupby('high_hisp_pop')[hmda_vars]

In [34]:
# Create the pivot table
# THIS GIVES US THE MEAN OF APPROVED LOANS IN ALL HIGH HISP TRACTS
# todo, how to find the values for each tract
pivot_table = pd.pivot_table(high_hispanic, 
                             index='high_hisp_pop', 
                             values=hmda_vars, 
                             aggfunc='mean')

# Rename the columns of the pivot table


# Print the result
print(pivot_table)

               num_applications  num_approved_loans
high_hisp_pop                                      
True                       9936                9936
